In [ ]:
! pip install pennylane-lightning[gpu]

In [1]:
import pennylane as qml
import pennylane.numpy as np
from pennylane.templates import ApproxTimeEvolution

In [3]:
dev = qml.device("default.qubit", wires=15)
# 0: ancillary, 1 ~ 14: LUMO+3 α β, LUMO+2 α β, ... , HOMO-2 α β

In [4]:
def Prep():
    qml.CNOT(wires=[0, 9])
    qml.CNOT(wires=[0, 10])    
    qml.CNOT(wires=[0, 11])    
    qml.CNOT(wires=[0, 12])    
    qml.CNOT(wires=[0, 13])    
    qml.CNOT(wires=[0, 14])
    angle = - 1 * np.arccos(np.sqrt(1/10))
    qml.CRY(angle, wires=[0, 4])
    qml.CNOT(wires=[4, 3])
    qml.CNOT(wires=[4, 13])
    qml.CNOT(wires=[4, 14])
    
def Prep_inv():
    qml.CNOT(wires=[4, 14])
    qml.CNOT(wires=[4, 13])
    qml.CNOT(wires=[4, 3])
    angle = np.arccos(np.sqrt(1/10))
    qml.CRY(angle, wires=[0, 4])
    qml.CNOT(wires=[0, 14])
    qml.CNOT(wires=[0, 13])    
    qml.CNOT(wires=[0, 12])    
    qml.CNOT(wires=[0, 11])    
    qml.CNOT(wires=[0, 10])    
    qml.CNOT(wires=[0, 9])    

In [5]:
def make_hamiltonian(distance):
    symbols, coordinates = (['Be', 'H', 'H'], np.array([0., 0., 0., 0., 0., -distance, 0., 0., distance]))
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, 
                                                mapping='jordan_wigner',#'bravyi_kitaev', 
                                                wires=[i + 1 for i in range(14)])
    return H

In [9]:
@qml.qnode(dev)
def circuit(distance, epsilon, time=2.0, trotterN=10):
    qml.Hadamard(wires=0)
    Prep()
    H = make_hamiltonian(distance)
    ApproxTimeEvolution(H, time, trotterN)
    Prep_inv()
    qml.PhaseShift(epsilon * time, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0])

In [ ]:
circuit(1.4, 3)